# Hugging Face TRL과 LoRA 어댑터를 활용해 LLM을 미세 조정하는 방법

이 노트북에서는 LoRA(Low-Rank Adaptation) 어댑터를 사용하여 대형 언어 모델을 효율적으로 파인튜닝하는 방법을 설명합니다. LoRA는 다음과 같은 특징을 가지는 파라미터 효율적인 미세 조정 기법입니다:
- 사전 학습 모델 가중치 고정
- 어텐션 레이어에 학습 가능한 저랭크 분해 행렬 추가
- 학습 가능한 파라미터 수가 최대 90%까지 감소
- 메모리를 효율적으로 사용하면서 모델 성능 유지

이 노트북에서 다루는 내용입니다:
1. 개발 환경 및 LoRA configuration 설정
2. 어댑터 학습을 위한 데이터셋 생성 및 준비
3. `trl`과 `SFTTrainer`를 활용한 LoRA 어댑터 기반 미세 조정
4. 모델 테스트 및 어댑터 병합(선택 사항)


## 1. 개발 환경 설정

첫 번째 단계는 trl, transformers, datasets을 포함한 Hugging Face 라이브러리와 Pytorch를 설치하는 것입니다. trl을 들어본 적이 없더라도 걱정하지 마세요. trl은 transformers와 datasets 위에 구축된 새로운 라이브러리로, 오픈 소스 LLM의 미세 조정과 RLHF, 정렬 작업을 쉽게 만들어줍니다.  


In [ ]:
# Google Colab에서 requirements 설치
# !pip install transformers datasets trl huggingface_hub

# Hugging Face 인증

from huggingface_hub import login

login()

# 허브 토큰을 HF_TOKEN 환경 변수로 설정해두면 편하게 사용할 수 있습니다.

## 2. 데이터셋 불러오기

In [13]:
# 예시 데이터셋 불러오기
from datasets import load_dataset

# TODO: path와 name 파라미터를 이용해 원하는 데이터셋 정의하기
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. LoRA와 함께 `trl`과 `SFTTrainer`를 사용한 LLM 미세 조정

`trl`에서 제공하는 [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)는 [PEFT](https://huggingface.co/docs/peft/en/index) 라이브러리를 통해 LoRA 어댑터와의 통합을 제공합니다. 이 설정의 주요 장점은 다음과 같습니다:

1. **메모리 효율성**: 
   - GPU 메모리에 어댑터 파라미터만 저장
   - 기본 모델 가중치는 고정되며 더 낮은 정밀도로 불러오기 가능
   - 서버가 아닌 소비자용 GPU에서 대형 모델 미세 조정 가능

2. **학습 기능**:
   - 최소한의 설정으로 기본 PEFT/LoRA 통합
   - 더 나은 메모리 효율을 위한 QLoRA(양자화된 LoRA) 지원

3. **어댑터 관리**:
   - 체크포인트 저장 시 어댑터 가중치도 저장
   - 어댑터를 기본 모델에 병합하는 기능 제공

이번 예제에서는 성능 저하 없이 메모리 사용량을 더욱 줄이기 위해 4비트 양자화를 결합한 LoRA를 사용하겠습니다. 설정은 몇 가지 단계만 거치면 됩니다:

1. LoRA configuration 설정(랭크, 알파, 드롭아웃)
2. PEFT configuration을 추가한 SFTTrainer 생성
3. 어댑터 가중치 학습 및 저장


In [ ]:
# 주요 라이브러리 불러오기
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# 모델과 토크나이저 불러오기
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# 대화 형식 설정
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# 미세 조정 결과를 저장하고 업로드하기 위한 이름 설정
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

`SFTTrainer`는 `peft`와의 기본 통합을 지원하기 때문에 LoRA와 같은 방법으로 LLM 미세 조정 작업을 매우 쉽고 효율적으로 진행할 수 있습니다. 우리는 `LoraConfig`를 생성하고 트레이너에게 제공하기만 하면 됩니다.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>연습: 미세 조정을 위한 LoRA 파라미터 정의하기</h2>
    <p>Hugging Face 허브에서 가져온 데이터셋으로 모델을 미세 조정 해보세요.</p> 
    <p><b>난이도</b></p>
    <p>🐢 일반적인 파라미터 값을 사용하여 임의로 미세 조정 진행하기</p>
    <p>🐕 파라미터를 조정하고 가중치와 편향 검토해보기</p>
    <p>🦁 파라미터를 조정하고 추론 결과 변화 확인하기</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO: LoRA 파라미터를 구성해보세요.
# r: LoRA 업데이트 행렬을 위한 랭크 차원원 (작을수록 압축률이 높아짐)
rank_dimension = 6
# lora_alpha: LoRA 레이어를 위한 스케일링 계수 (높을수록 모델이 새로운 데이터에 잘 적응함)
lora_alpha = 8
# lora_dropout: LoRA 레이어의 드롭아웃 확률 (과적합 방지)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # 랭크 차원 - 일반적으로 4-32 사이의 값 사용
    lora_alpha=lora_alpha,  # LoRA 스케일링 계수 - 일반적으로 랭크 차원의 2배
    lora_dropout=lora_dropout,  # LoRA 레이어의의 드롭아웃 확률
    bias="none",  # LoRA 편향 유형 - 해당 편향은 학습 중 업데이트됨됨
    target_modules="all-linear",  # LoRA를 적용할 모듈듈
    task_type="CAUSAL_LM",  # 모델 구조에 맞는 태스크 유형
)

학습을 시작하기 전 사용하고자 하는 하이퍼파라미터(`TrainingArguments`) 값을 설정해야 합니다.

In [ ]:
# 학습을 위한 configuration
# QLoRA 논문에 기반한 하이퍼파라미터 설정
args = SFTConfig(
    # 출력 설정
    output_dir=finetune_name,  # 모델 체크포인트 저장 디렉토리
    # 학습 기간
    num_train_epochs=1,  # 학습 에포크 수
    # 배치 크기 설정
    per_device_train_batch_size=2,  # GPU당 배치 크기
    gradient_accumulation_steps=2,  # 더 큰 배치 크기 효과를 위한 경사 누적
    # 메모리 최적화
    gradient_checkpointing=True,  # 학습 속도는 느려지지만 메모리 절약 가능
    # 옵티마이저 설정
    optim="adamw_torch_fused",  # 효율성을 위해 fused AdamW 사용
    learning_rate=2e-4,  # 학습률 (QLoRA 논문 참고)
    max_grad_norm=0.3,  # 경사 클리핑 임곗값
    # 학습률 스케줄링
    warmup_ratio=0.03,  # 학습률을 0에서 목표 값까지 선형적으로 증가시키는 웜업을 위한 전체 학습 스텝의 비율
    lr_scheduler_type="constant",  # 웜업 후 일정한 학습률 유지
    # 로깅 및 저장
    logging_steps=10,  # 매 N 스텝마다 지표 기록
    save_strategy="epoch",  # 매 에포크마다 체크포인트 저장
    # 정밀도 설정
    bf16=True,  # bfloat16 정밀도 사용
    # 통합 설정
    push_to_hub=False,  # Hugging Face 허브로 내보내지 않음
    report_to="none",  # 외부 로깅 비활성화
)

이제 모델 학습을 위한 `SFTTrainer` 생성에 필요한 요소를 모두 갖췄습니다.

In [ ]:
max_seq_length = 1512  # 모델과 데이터셋 패킹을 위한 최대 시퀀스 길이

# LoRA configuration으로 SFTTrainer 생성
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # LoRA configuration
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    tokenizer=tokenizer,
    packing=True,  # 효율성을 위해 입력 패킹 활성화
    dataset_kwargs={
        "add_special_tokens": False,  # 템플릿에서 추가 토큰 처리
        "append_concat_token": False,  # 추가 구분자 없음
    },
)

`Trainer` 객체의 `train()` 메서드를 호출해 학습을 시작합니다. 학습 루프가 시작되고 3 에포크 동안 모델이 학습됩니다. 지금은 PEFT 방법을 쓰고 있기 때문에 전체 모델을 저장하지 않고 적응된 모델 가중치만 저장합니다.

In [ ]:
# 학습이 시작되면 허브와 출력 디렉토리에 모델 자동 저장
trainer.train()

# 모델 저장
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

`g5.2xlarge` 인스턴스에서 플래시 어텐션을 적용해 15,000개의 샘플을 3 에포크 동안 학습하는 데 걸린 시간은 4시간 14분 36초였습니다. 해당 인스턴스 비용은 시간당 1.21달러로, 총 비용은 약 `5.3달러`에 불과합니다.

### 기존 모델과 LoRA 어댑터 병합

LoRA를 사용할 때 기본 모델은 고정된 상태로 두고 어댑터 가중치만 학습합니다. 학습 과정에서는 전체 모델이 아닌 2-10MB 크기의 가벼운 어댑터 가중치만 저장합니다. 하지만 배포할 경우 다음과 같은 이유로 어댑터를 기본 모델과 병합하는 것이 더 나을 수 있습니다:

1. **간소화된 배포**: 기본 모델과 어댑터를 따로 관리하지 않고 단일 모델로 처리 가능
2. **추론 속도**: 어댑터 계산에 따른 추가적인 오버헤드 없음
3. **프레임워크 호환성**: 서빙 프레임워크와의 호환성 향상


In [ ]:
from peft import AutoPeftModelForCausalLM


# CPU에서 PEFT 모델 불러오기
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# LoRA와 기본 모델 병합 후 저장
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. 모델 테스트 및 추론 수행

학습이 끝난 후 모델을 테스트 할 것입니다. 원본 데이터셋에서 다양한 예제를 불러온 뒤, 해당 예제를 활용한 간단한 루프에서 정확도를 평가 지표로 삼아 모델을 평가합니다.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>추가 연습: LoRA 어댑터 불러오기</h2>
    <p>예제 노트북에서 배운 내용을 활용하여 학습된 LoRA 어댑터를 불러오고 추론에 사용하세요.</p> 
</div>

In [30]:
# 메모리 확보
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# PEFT 어댑터가 있는 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

몇 가지 프롬프트 예제로 테스트해보고 모델이 어떻게 작동하는지 확인해봅시다.

In [34]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)